In [1]:
# Magics for the Jupyter notebook
%matplotlib inline

In [2]:
import os
data_path = "C:/Users/Axeld/Desktop/SenNet/blood-vessel-segmentation/train/"

# Imports

In [3]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm
import cv2
import numpy as np

from utils.mesh import make_mesh

# Make Meshes of the Vasculatures

In [4]:

for min_area in [0, 20, 50, 100, 200]:
    for dataset in os.listdir(data_path):
        dataset_path = os.path.join(data_path, dataset)
        if "labels" not in os.listdir(dataset_path):
            continue
        if "voi" in dataset:
            continue
        masks_path = os.path.join(data_path, dataset, "labels")

        os.makedirs("meshes", exist_ok=True)

        #if os.path.exists(f"meshes/{dataset}_area{min_area}.obj"):
        #    continue

        f = open(f"meshes/{dataset}_area{min_area}.obj", "w")

        make_mesh(masks_path, min_area, f"meshes/{dataset}_area{min_area}.obj")
        break
    break


  0%|          | 0/2279 [00:00<?, ?it/s]

100%|██████████| 2279/2279 [00:58<00:00, 39.06it/s]


# Make a video of the volume

In [ ]:
scale = 2
for dataset in os.listdir(data_path):
    dataset_path = os.path.join(data_path, dataset)
    if "labels" not in os.listdir(dataset_path):
        continue
    if "images" not in os.listdir(dataset_path):
        continue
    if "voi" in dataset:
        continue
    images_path = os.path.join(data_path, dataset, "images")
    masks_path = os.path.join(data_path, dataset, "labels")

    files = os.listdir(masks_path)
    files.sort(key=lambda x: int(x.split(".")[0]))

    video = None

    for file in tqdm(files):

        i = int(file.split(".")[0])

        if i % scale != 0:
            continue
        
        img = cv2.imread(os.path.join(images_path, file), cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(os.path.join(masks_path, file), cv2.IMREAD_GRAYSCALE)

        img = cv2.equalizeHist(img)
        
        # Decrease the resolution
        img = cv2.resize(img, (img.shape[0] // scale, img.shape[1] // scale), interpolation=cv2.INTER_NEAREST)
        mask = cv2.resize(mask, (mask.shape[0] // scale, mask.shape[1] // scale), interpolation=cv2.INTER_NEAREST)

        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        mask = np.concatenate([mask[:, :, None], mask[:, :, None], mask[:, :, None]], axis=2)
        
        # Make a red mask
        mask[:, :, 0] = 0
        mask[:, :, 1] = 0

        overlay = cv2.addWeighted(img, 0.5, mask, 0.5, 0)

        encoding = cv2.VideoWriter_fourcc(*'mp4v')
        if video is None:
            video = cv2.VideoWriter(f"{dataset}.mp4", encoding, 10, (img.shape[1], img.shape[0]))

        video.write(overlay)

    video.release()

  0%|          | 0/2279 [00:00<?, ?it/s]

100%|██████████| 1035/1035 [00:53<00:00, 19.31it/s]
